Using
* Add in FFN - Not using Multiply
* MaxPool1D
* using Dense in Transformer layer

Failed atempts
* In Inception layer, use DepthwiseConv1D in place of MaxPool1D
* In Inception layer, use AveragePool1D in place of MaxPool1D
* using hamming_window in Fourier transform


# Import Lib
Neural machine translation with a Transformer and Keras - https://www.tensorflow.org/text/tutorials/transformer#the_transformer

In [ ]:
#!pip install tensorflow_text

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = [15,8]
import seaborn as sns
import plotly.graph_objects as go

In [ ]:
import tensorflow as tf
from tensorflow.keras import Sequential, Model
from tensorflow.keras.layers import Embedding, Layer, MultiHeadAttention, LayerNormalization, Conv1D, MaxPool1D
from tensorflow.keras.layers import Add, Dense, Dropout, Flatten, Concatenate, BatchNormalization, Multiply, DepthwiseConv1D
from tensorflow.keras.optimizers.schedules import LearningRateSchedule
from tensorflow.math import rsqrt, minimum
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau, Callback
from tensorflow.keras.models import load_model, save_model
#from tensorflow.keras.ops import round as tf_round
from tensorflow.keras.regularizers import l2, l1
from tensorflow.keras.backend import set_value
from tensorflow.signal import stft, hann_window

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, r2_score, mean_absolute_error, mean_squared_error

In [ ]:
from tensorflow.keras import initializers

initializer_for_relu = initializers.HeNormal() # For layers with activation function Relu
initializer_for_sigmoid = initializers.GlorotNormal() # For layers with activation function Sigmoid

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# Data Preprocessing

## Import Data

In [ ]:
# Mount google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = '/content/drive/MyDrive/MyColabProject/Data'
data_df = pd.read_csv(path+'/raw_nsepy_inp512_differencedVal_fourierTransform.csv')
data_df.head(3)

0     1    2     3     4     5     6     7     8    9  ...   503   504  \
0 -4.45  6.45 -3.1  3.00  3.10 -2.55 -3.25  9.45  1.55 -5.2  ... -3.25  5.70   
1  6.45 -3.10  3.0  3.10 -2.55 -3.25  9.45  1.55 -5.20 -4.0  ...  5.70 -5.45   
2 -3.10  3.00  3.1 -2.55 -3.25  9.45  1.55 -5.20 -4.00  1.5  ... -5.45  3.20   

    505   506   507   508   509   510   511   512  
0 -5.45  3.20 -4.45 -2.15 -1.05 -1.10  3.05  4.35  
1  3.20 -4.45 -2.15 -1.05 -1.10  3.05  4.35  2.05  
2 -4.45 -2.15 -1.05 -1.10  3.05  4.35  2.05  1.70  

[3 rows x 513 columns]

In [ ]:
data_df.shape

(41412, 513)

## Train Test Split
* symbol =
           'BPCL','POWERGRID','NTPC','SUNPHARMA','TATACONSUM','ONGC','HINDALCO','ICICIBANK','SBIN','BHARTIARTL',
           'WIPRO','ITC','AXISBANK','JSWSTEEL','COALINDIA','HDFCLIFE','TATAMOTORS'
* Open, high, low, close  = 609
* Complete set = 2436, 4872, 9744
* in use = 19488

In [ ]:
data_df.iloc[:2400,256:].shape

(2400, 257)

In [ ]:
data_df.iloc[:2400,256:].head(3)

256   257   258   259  260   261   262   263   264   265  ...   503   504  \
0  2.75 -0.40 -1.65 -0.85 -1.0  0.30  3.90  7.85  2.75  4.30  ... -3.25  5.70   
1 -0.40 -1.65 -0.85 -1.00  0.3  3.90  7.85  2.75  4.30 -5.35  ...  5.70 -5.45   
2 -1.65 -0.85 -1.00  0.30  3.9  7.85  2.75  4.30 -5.35 -0.25  ... -5.45  3.20   

    505   506   507   508   509   510   511   512  
0 -5.45  3.20 -4.45 -2.15 -1.05 -1.10  3.05  4.35  
1  3.20 -4.45 -2.15 -1.05 -1.10  3.05  4.35  2.05  
2 -4.45 -2.15 -1.05 -1.10  3.05  4.35  2.05  1.70  

[3 rows x 257 columns]

In [ ]:
data_q1 = data_df.iloc[:,:129].copy()
data_q2 = data_df.iloc[:,128:257].copy()
data_q3 = data_df.iloc[:,256:385].copy()
data_q4 = data_df.iloc[:,384:].copy()

data_df_merged = pd.DataFrame()
data_df_merged = pd.concat([pd.DataFrame(data_q1.values),pd.DataFrame(data_q2.values),pd.DataFrame(data_q3.values),pd.DataFrame(data_q4.values)],
                           ignore_index=True)
print(data_df_merged.shape)
data_df_merged.head(3)

(165648, 129)


0     1    2     3     4     5     6     7     8    9    ...   119   120  \
0 -4.45  6.45 -3.1  3.00  3.10 -2.55 -3.25  9.45  1.55 -5.2  ... -7.65 -1.25   
1  6.45 -3.10  3.0  3.10 -2.55 -3.25  9.45  1.55 -5.20 -4.0  ... -1.25  0.00   
2 -3.10  3.00  3.1 -2.55 -3.25  9.45  1.55 -5.20 -4.00  1.5  ...  0.00  8.00   

    121   122   123   124   125  126  127  128  
0  0.00  8.00 -6.05  1.80 -0.75 -0.9 -0.1 -2.8  
1  8.00 -6.05  1.80 -0.75 -0.90 -0.1 -2.8 -1.5  
2 -6.05  1.80 -0.75 -0.90 -0.10 -2.8 -1.5  0.5  

[3 rows x 129 columns]

In [ ]:
data_df_merged.tail(3)

0     1     2     3     4     5     6    7      8      9    ...  \
165645  15.15  -1.0  12.4  3.40  3.20 -5.15  4.85 -4.7   6.00   6.00  ...   
165646  -1.00  12.4   3.4  3.20 -5.15  4.85 -4.70  6.0   6.00 -22.05  ...   
165647  12.40   3.4   3.2 -5.15  4.85 -4.70  6.00  6.0 -22.05  -5.75  ...   

         119    120    121    122   123   124   125    126    127   128  
165645  1.35   1.00   4.65 -12.95  26.9  -1.0 -14.1  -9.40 -37.60  4.75  
165646  1.00   4.65 -12.95  26.90  -1.0 -14.1  -9.4 -37.60   4.75 -5.25  
165647  4.65 -12.95  26.90  -1.00 -14.1  -9.4 -37.6   4.75  -5.25 -3.50  

[3 rows x 129 columns]

In [ ]:
data_df_merged.shape

(165648, 129)

In [ ]:
inp_len = 128 # Length of the input
out_len = 1 # Length of Output and Context
#inp_rows = int(data_df_merged.shape[0]/3)
random_int = 1
inp_rows

55216

In [ ]:
#data_df_sampled = data_df_merged.sample(inp_rows, random_state=random_int, ignore_index=True)
#print(data_df_sampled.shape)

(55216, 129)


In [ ]:
prediction_percentage = 0.1 #0.025
train_df, test_df = train_test_split(data_df_merged, test_size=prediction_percentage, random_state=random_int, shuffle=True)

In [ ]:
print('Train shape',train_df.shape)
print('Test shape',test_df.shape)

Train shape (149083, 129)
Test shape (16565, 129)


In [ ]:
train_df, val_df = train_test_split(train_df, test_size=0.25, random_state=random_int, shuffle=True)

In [ ]:
print('Train shape',train_df.shape)
print('Validation shape',val_df.shape)
print('Test shape',test_df.shape)

Train shape (111812, 129)
Validation shape (37271, 129)
Test shape (16565, 129)


Keras Model.fit training expects (inputs, labels) pairs. The inputs is tokenized sequences. The labels are the same sequences shifted by 1. This shift is so that at each location input sequence, the label in the next token.

In [ ]:
X_train = train_df.drop(columns=train_df.columns[inp_len:].values).values
y_train = train_df.drop(columns=train_df.columns[:inp_len].values).values

X_val = val_df.drop(columns=val_df.columns[inp_len:].values).values
y_val = val_df.drop(columns=val_df.columns[:inp_len].values).values

X_test = test_df.drop(columns=test_df.columns[inp_len:].values).values
y_test = test_df.drop(columns=test_df.columns[:inp_len].values).values


In [ ]:
print('X Train shape',X_train.shape)
print('Y Train shape',y_train.shape)

print('X Validation shape',X_val.shape)
print('Y Validation shape',y_val.shape)

print('X Test shape',X_test.shape)
print('Y Test shape',y_test.shape)

X Train shape (111812, 128)
Y Train shape (111812, 1)
X Validation shape (37271, 128)
Y Validation shape (37271, 1)
X Test shape (16565, 128)
Y Test shape (16565, 1)


In [ ]:
X_train_f = X_train.astype(np.float32)
y_train_t = tf.convert_to_tensor(y_train)

X_val_f = X_val.astype(np.float32)
y_val_t = tf.convert_to_tensor(y_val)

X_test_f = X_test.astype(np.float32)
y_test_t = tf.convert_to_tensor(y_test)

In [ ]:
X_train_f[0]

array([ -2.1 ,  -1.35,   2.55,   0.2 ,   4.05,   0.45,   0.1 ,  12.2 ,
        -2.4 ,  12.5 ,   2.9 ,   1.65,  -4.35,   7.5 ,  -0.8 ,   6.7 ,
        -7.4 ,  -1.7 ,  -3.35,  -4.95,   5.2 ,  -1.4 ,   5.7 ,   0.7 ,
         4.6 ,  -4.7 ,   2.  ,  -1.6 ,  -0.85,   7.6 ,  -6.15,   0.5 ,
        -0.8 ,   4.75,   0.85,  -1.5 ,   1.2 ,  -0.45,  -1.85,   0.1 ,
         5.8 ,  -2.5 ,  37.7 ,  -1.75,   0.4 ,  11.3 ,  -2.  ,  -5.35,
         1.4 ,   3.05,  -1.8 ,   6.05,   2.5 ,   7.1 ,  -3.75,   1.25,
        -0.2 ,  -3.1 ,   1.75,  -1.9 ,  -0.8 ,   2.8 ,  -3.9 ,   0.85,
         0.1 ,   6.5 ,  -0.5 ,  -2.6 ,  -5.9 ,  -1.  ,  -0.8 ,  -2.8 ,
         2.1 ,   0.9 ,  -0.3 ,  -3.75,   3.3 ,   2.05,   9.2 ,  -6.7 ,
         2.  ,   3.55,   1.4 ,   1.4 ,   7.5 ,  13.55,   0.5 ,   5.9 ,
        -1.6 ,   6.35,  -6.2 ,   1.95,   1.4 , -10.4 ,   9.9 ,  -2.35,
         0.45,   1.25,   0.05,  21.7 ,  -5.45,   6.45,  23.9 ,  15.35,
         4.25,  -4.3 , -25.8 ,  -7.9 ,  -2.4 ,   4.5 ,   3.4 ,  -3.7 ,
      

In [ ]:
y_train[0]

array([-2.05])

In [ ]:
y_train_t[0]

<tf.Tensor: shape=(1,), dtype=float64, numpy=array([-2.05])>

## Testing tf.signal.stft

In [ ]:
spectrogram1 = tf.signal.stft(signals=X_train_f[0],
                              frame_length=512,
                              frame_step=1)

print(X_train[0].shape[0])
print(spectrogram1.shape)
print(spectrogram1)

128
(0, 257)
tf.Tensor([], shape=(0, 257), dtype=complex64)


In [ ]:
spectrogram2 = tf.signal.stft(X_train_f[0], frame_length=512, frame_step=256)

print(X_train[0].shape[0])
print(spectrogram2.shape)
print(spectrogram2)

128
(0, 257)
tf.Tensor([], shape=(0, 257), dtype=complex64)


In [ ]:
spectrogram3 = tf.signal.stft(X_train_f[0], frame_length=256, frame_step=128)

print(X_train[0].shape[0])
print(spectrogram3.shape)
print(spectrogram3)

128
(0, 129)
tf.Tensor([], shape=(0, 129), dtype=complex64)


In [ ]:
spectrogram4 = tf.signal.stft(X_train_f[0], frame_length=16, frame_step=8)

print(X_train[0].shape[0])
print(spectrogram4.shape)
print(spectrogram4)

128
(15, 9)
tf.Tensor(
[[ 28.058962  +0.00000000e+00j -20.63869   +2.18879461e-01j
   11.98763   -1.86132288e+00j  -5.839611  +4.91298151e+00j
   -5.3669415 -1.40579224e+00j  13.898495  -6.17310953e+00j
  -16.48763   +3.89976835e+00j  22.179806  +5.93278790e+00j
  -27.52508   +0.00000000e+00j]
 [ -2.1056252 +0.00000000e+00j   6.1707644 -8.55994129e+00j
   -5.3699207 +1.34032087e+01j   3.9137983 -1.02501583e+01j
   -4.692194  -3.46133709e-02j   4.0955915 +1.79806471e+00j
  -10.280079  +8.22998714e+00j  15.419846  -1.56117163e+01j
  -16.409988  +0.00000000e+00j]
 [  6.8066063 +0.00000000e+00j  -5.6117387 -4.05283928e+00j
    0.5601119 +9.31721592e+00j  -0.18571842-5.64927864e+00j
    1.5938807 +1.26627815e+00j   2.2479525 -6.65308666e+00j
    4.289888  +3.82102084e+00j -14.850496  +7.04335213e+00j
   17.10563   +0.00000000e+00j]
 [  3.3326774 +0.00000000e+00j  -2.6691346 +1.00122690e+00j
   -0.06235772-4.60078764e+00j   0.06676769+9.01872540e+00j
    3.8768659 -1.05517216e+01j  -4.8955  

# Model Architecture <br>
* The querys is what you're trying to find.
* The keys what sort of information the dictionary has.
* The value is that information.


<b>fft_length and frame_length</b>
* The number of frequency bins depends on the FFT size (fft_length), which is often set to the next power of 2 that is greater than or equal to frame_length
* example, if frame_lenght = 39, Next power of 2 greater than 39 is 64. Setting fft_length = 64 would allow the FFT algorithm to work more efficiently by padding the input to a length of 64.
* or choose frame_lenght any of 2, 4, 8, 16, 32, 64, ... <br>

<b>frame_step</b>
* To choose frame_step, 50% Overlap (a common choice): To achieve 50% overlap, set frame_step to half the frame_length: frame_step = frame_length/2
* example, if frame_length=8, then frame_step=8/2 = 4. This will give you overlapping frames, which improves frequency resolution.<br>

<b>Frequency_bins v/s time_bins</b>
*  have more Frequency_bins if identifying a long-term uptrend or downtrend based on weekly or monthly stock price data, or detecting cyclic behavior like seasonality.
* have more time_bins if Detecting intraday trends or anomalies (like a sudden price surge due to a news release).

In [ ]:
# Hyperparameters for Fourier Transform
sample_signal_length = [128, 64, 32, 16] # Length of the input time series
sample_frame_length = [64, 32, 16, 8] # window size for Fourier Transform
sample_frame_step = [32, 16, 8, 4] # Hop size for Fourier Transform
sample_inception_lyr_count = len(sample_signal_length)
sample_max_signal_length = max(sample_signal_length)
sample_min_signal_length = min(sample_signal_length)

# Hyperparameters for Attention Layer and DNN Layer
sample_num_layers = [1, 2, 4, 6] # number of TransformerEncoderLayer layers (Original paper = 6)
sample_num_heads = [1, 2, 4, 6] # number of self-attention heads in the MultiheadAttention layer (Original paper = 8)
sample_dropout_rate = 0.2 # Dropout rate
sample_regularizer_rate = 0.005 # Use to regularizer the weights in attention model

# Extra Parameters
sample_index_for_testing = 0

In [ ]:
int(tf.math.floor((sample_frame_length[sample_index_for_testing]/2) +1))

33

In [ ]:
sample_frame_length[sample_index_for_testing]

64

In [ ]:
# The STFT output shape can be defined as:
# Output Shape = (batch_size, 𝐹, 𝑇)
sample_time_bins = list()
sample_frequency_bins = list()
for i in np.arange(sample_inception_lyr_count):
  print(i)
  # Number of Time Frames (T)
  sample_time_bins.append(int(((sample_signal_length[i] - sample_frame_length[i])/sample_frame_step[i] )+1 ))
  print('Time Bins =',sample_time_bins[i])

  # Number of Frequency Bins (F)
  sample_frequency_bins.append(int(tf.math.floor((sample_frame_length[i]/2) +1)))
  print('Frequency Bins =',sample_frequency_bins[i])

  print('output_shape=[batch_size, time_bins, frequency_bins]')
  print('Output shape = (batch_size,',sample_time_bins[i],',',sample_frequency_bins[i],')\n')

0
Time Bins = 3
Frequency Bins = 33
output_shape=[batch_size, time_bins, frequency_bins]
Output shape = (batch_size, 3 , 33 )

1
Time Bins = 3
Frequency Bins = 17
output_shape=[batch_size, time_bins, frequency_bins]
Output shape = (batch_size, 3 , 17 )

2
Time Bins = 3
Frequency Bins = 9
output_shape=[batch_size, time_bins, frequency_bins]
Output shape = (batch_size, 3 , 9 )

3
Time Bins = 3
Frequency Bins = 5
output_shape=[batch_size, time_bins, frequency_bins]
Output shape = (batch_size, 3 , 5 )



In [ ]:
tf.math.sqrt(tf.cast(65, dtype=tf.bfloat16))*3

<tf.Tensor: shape=(), dtype=bfloat16, numpy=24.25>

In [ ]:
tf.math.sqrt(tf.cast(33, dtype=tf.bfloat16))*3

<tf.Tensor: shape=(), dtype=bfloat16, numpy=17.25>

In [ ]:
tf.math.sqrt(tf.cast(17, dtype=tf.bfloat16))*3

<tf.Tensor: shape=(), dtype=bfloat16, numpy=12.375>

In [ ]:
tf.math.sqrt(tf.cast(9, dtype=tf.bfloat16))*3

<tf.Tensor: shape=(), dtype=bfloat16, numpy=9>

In [ ]:
tf.math.sqrt(tf.cast(5, dtype=tf.bfloat16))*3

<tf.Tensor: shape=(), dtype=bfloat16, numpy=6.6875>

In [ ]:
data_512_vals = data_df.iloc[:,:-1].values
data_512_vals

array([[ -4.45,   6.45,  -3.1 , ...,  -1.05,  -1.1 ,   3.05],
       [  6.45,  -3.1 ,   3.  , ...,  -1.1 ,   3.05,   4.35],
       [ -3.1 ,   3.  ,   3.1 , ...,   3.05,   4.35,   2.05],
       ...,
       [  4.4 ,   6.  ,  -6.1 , ..., -14.1 ,  -9.4 , -37.6 ],
       [  6.  ,  -6.1 ,  -2.55, ...,  -9.4 , -37.6 ,   4.75],
       [ -6.1 ,  -2.55, -15.25, ..., -37.6 ,   4.75,  -5.25]])

In [ ]:
data_512_vals[:,-sample_frame_length[sample_index_for_testing]:].shape

(41412, 64)

In [ ]:
data_512_vals[:,-sample_frame_length[sample_index_for_testing]:]

array([[  1.1 ,   6.8 ,   2.5 , ...,  -1.05,  -1.1 ,   3.05],
       [  6.8 ,   2.5 ,  -2.9 , ...,  -1.1 ,   3.05,   4.35],
       [  2.5 ,  -2.9 ,   4.5 , ...,   3.05,   4.35,   2.05],
       ...,
       [ -5.25,  25.25,  -9.6 , ..., -14.1 ,  -9.4 , -37.6 ],
       [ 25.25,  -9.6 ,  10.  , ...,  -9.4 , -37.6 ,   4.75],
       [ -9.6 ,  10.  ,  11.25, ..., -37.6 ,   4.75,  -5.25]])

In [ ]:
data_512_vals[:,-sample_frame_length[1]:].shape

(41412, 32)

In [ ]:
data_512_vals[:,-sample_frame_length[1]:]

array([[  1.15,  -4.15,  -1.7 , ...,  -1.05,  -1.1 ,   3.05],
       [ -4.15,  -1.7 ,   3.15, ...,  -1.1 ,   3.05,   4.35],
       [ -1.7 ,   3.15,   0.75, ...,   3.05,   4.35,   2.05],
       ...,
       [ -3.  ,  -0.15, -14.85, ..., -14.1 ,  -9.4 , -37.6 ],
       [ -0.15, -14.85,  13.35, ...,  -9.4 , -37.6 ,   4.75],
       [-14.85,  13.35,   9.05, ..., -37.6 ,   4.75,  -5.25]])

## The Fourier Transform layer

The output is a 2D array of shape (n_freqs, n_times), where n_freqs is the number of frequency bins (same as the length of f), and n_times is the number of time segments (same as the length of t).
* Rows of Zxx correspond to different frequencies (i.e., values in the f array).
* Columns of Zxx correspond to different time segments (i.e., values in the t array).
* The magnitude of the complex numbers in Zxx (np.abs(Zxx)) represents the strength or amplitude of each frequency component at that time segment.
* The phase of the complex numbers in Zxx (np.angle(Zxx)) represents the phase information of the signal at that frequency and time.
*  A larger window (windows_size, hops) gives better frequency resolution but poorer time resolution. You may need to experiment with different values based on your data.

* tf.py_function allows TensorFlow to execute scipy_stft_fn, passing a tensor as input, converting it to a NumPy array internally, and returning a tensor that is compatible with TensorFlow.
* In the build method of the FourierTransform class, the shape of self.kernel should be a tuple, but it is set as a single value (self.windows_size). self.windows_size should be wrapped in parentheses to specify the shape correctly, assuming it's a 1D kernel.

In [ ]:
class FourierTransform(Layer):
  def __init__(self, signal_length, frame_length, frame_step):
    super(FourierTransform, self).__init__()
    self.signal_length = signal_length
    self.frame_length = frame_length
    self.frame_step = frame_step

  def build(self, input_shape):
    # Define weights
    self.kernel = self.add_weight(
        shape=(self.signal_length,), # the use of ',' after self.signal_len is a must, read the above mentioned comments point-2
        initializer=initializer_for_relu,
        trainable=False
    )

  def call(self, x):
    # window_gen = hann_window(self.windows_size)  # symmetric Gaussian window
    # Convert the waveform to a spectrogram via a STFT.
    spectrogram = tf.signal.stft(signals=x, frame_length=self.frame_length, frame_step=self.frame_step)
    magnitude_x = tf.math.abs(spectrogram)
    angle_x = tf.math.angle(spectrogram) # Disable it if using only magnitude as output
    #magnitude_x = tf_round(magnitude_x, 4)
    #angle_x = tf_round(angle_x, 4) # Disable it if using only magnitude as output
    return magnitude_x , angle_x

In [ ]:
# Create an Embedding Object
sft_layer = FourierTransform(signal_length=sample_signal_length[sample_index_for_testing],
                             frame_length=sample_frame_length[sample_index_for_testing],
                             frame_step=sample_frame_step[sample_index_for_testing])
sft_layer.build(X_train_f[:3].shape)
# Calling the function
out_sft_mag, out_sft_ang = sft_layer(X_train_f[:3])

In [ ]:
print(out_sft_mag.shape)

(3, 3, 33)


In [ ]:
X_train_f[0,:]

array([ -2.1 ,  -1.35,   2.55,   0.2 ,   4.05,   0.45,   0.1 ,  12.2 ,
        -2.4 ,  12.5 ,   2.9 ,   1.65,  -4.35,   7.5 ,  -0.8 ,   6.7 ,
        -7.4 ,  -1.7 ,  -3.35,  -4.95,   5.2 ,  -1.4 ,   5.7 ,   0.7 ,
         4.6 ,  -4.7 ,   2.  ,  -1.6 ,  -0.85,   7.6 ,  -6.15,   0.5 ,
        -0.8 ,   4.75,   0.85,  -1.5 ,   1.2 ,  -0.45,  -1.85,   0.1 ,
         5.8 ,  -2.5 ,  37.7 ,  -1.75,   0.4 ,  11.3 ,  -2.  ,  -5.35,
         1.4 ,   3.05,  -1.8 ,   6.05,   2.5 ,   7.1 ,  -3.75,   1.25,
        -0.2 ,  -3.1 ,   1.75,  -1.9 ,  -0.8 ,   2.8 ,  -3.9 ,   0.85,
         0.1 ,   6.5 ,  -0.5 ,  -2.6 ,  -5.9 ,  -1.  ,  -0.8 ,  -2.8 ,
         2.1 ,   0.9 ,  -0.3 ,  -3.75,   3.3 ,   2.05,   9.2 ,  -6.7 ,
         2.  ,   3.55,   1.4 ,   1.4 ,   7.5 ,  13.55,   0.5 ,   5.9 ,
        -1.6 ,   6.35,  -6.2 ,   1.95,   1.4 , -10.4 ,   9.9 ,  -2.35,
         0.45,   1.25,   0.05,  21.7 ,  -5.45,   6.45,  23.9 ,  15.35,
         4.25,  -4.3 , -25.8 ,  -7.9 ,  -2.4 ,   4.5 ,   3.4 ,  -3.7 ,
      

In [ ]:
# Inspecting the time-component when frequency-component = 3
out_sft_mag[0,1,:]

<tf.Tensor: shape=(33,), dtype=float32, numpy=
array([24.612616 , 13.424038 , 24.952284 , 12.635899 ,  4.5033374,
       15.271859 , 32.69937  ,  7.6079655,  8.076834 , 23.078619 ,
       22.729904 , 21.037903 ,  5.3654156,  8.369405 ,  9.588105 ,
       16.401915 , 32.424694 , 11.470016 , 20.84756  , 17.233286 ,
       21.072186 , 17.138025 ,  6.454736 , 24.036623 , 18.3824   ,
       20.425783 , 13.643231 , 15.524836 , 29.33502  , 21.072897 ,
       29.542208 , 19.76541  ,  5.4224644], dtype=float32)>

In [ ]:
# Inspecting the time-component when frequency-component = 12
out_sft_mag[0,0,:]

<tf.Tensor: shape=(33,), dtype=float32, numpy=
array([49.491127, 34.274242, 32.140053, 26.09904 , 36.065178, 18.139193,
       43.519726, 40.450737, 19.078407, 35.295578, 26.107515,  6.092731,
        9.741023, 20.151335, 32.453144, 26.026157, 26.274452, 15.675907,
       26.524143, 35.270298, 51.876812, 42.312157, 28.266752, 49.463833,
       53.89004 , 33.82923 , 26.100286,  9.199308, 56.656548, 70.73444 ,
       20.886665, 29.973215, 16.132872], dtype=float32)>

In [ ]:
# Inspecting the frequency-component when time-component = 3
out_sft_mag[0,:,1]

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([34.274242, 13.424038, 50.459362], dtype=float32)>

In [ ]:
# Inspecting the frequency-component when time-component = 12
out_sft_mag[0,:,4]

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([36.065178 ,  4.5033374, 76.93256  ], dtype=float32)>

In [ ]:
print(out_sft_ang.shape)

(3, 3, 33)


In [ ]:
# Inspecting the time-component when frequency-component = 3
out_sft_ang[0,0,:]

<tf.Tensor: shape=(33,), dtype=float32, numpy=
array([ 0.        ,  2.2116117 , -1.9928472 , -0.16184643,  2.5189228 ,
       -1.778925  ,  0.10555109,  2.6825914 , -1.8654624 ,  0.2909191 ,
       -2.7889202 , -0.8824661 ,  0.9491725 ,  2.4041626 , -0.9560039 ,
        1.728126  , -2.2206497 , -0.19845824,  1.2240497 ,  3.1012075 ,
       -0.9288144 ,  1.412665  , -2.9566808 , -1.2170975 ,  1.4328308 ,
       -2.6662464 , -0.14104566,  0.7849196 ,  3.1076412 , -0.1559727 ,
        2.292562  , -2.3646047 ,  0.        ], dtype=float32)>

In [ ]:
# Inspecting the time-component when frequency-component = 12
#out_sft_ang[0,6,:]

In [ ]:
# Inspecting the frequency-component when time-component = 3
out_sft_ang[0,:,1]

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([ 2.2116117 , -0.44040293, -2.7826738 ], dtype=float32)>

In [ ]:
# Inspecting the frequency-component when time-component = 12
out_sft_ang[0,:,4]

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([ 2.5189228,  0.2925817, -1.7081486], dtype=float32)>

## Custom Attention Block

In [ ]:
class BaseAttention(Layer):
  def __init__(self, frequency_bins, time_bins, **kwargs):
    super().__init__()
    self.mha = MultiHeadAttention(key_dim = frequency_bins,
                                  kernel_initializer = initializer_for_relu,
                                  **kwargs)
    self.layernorm = LayerNormalization()
    self.add = Add()
    self.frequency_bins = frequency_bins
    self.time_bins = time_bins

  def build(self, input_shape):
    # Define weights
    self.kernel = self.add_weight(
        shape=(self.time_bins, self.frequency_bins),
        initializer=initializer_for_relu,
        trainable=True
    )

## Self-attention layer - Magnitude and Angle

* tf.ensure_shape Updates the shape of a tensor and checks at runtime that the shape holds.
* When executed, this operation asserts that the input tensor x's shape is compatible with the shape argument.

In [ ]:
class SelfAttention(BaseAttention):
  def __init__(self, frequency_bins, time_bins, **kwargs):
      # Call the parent class (BaseAttention) constructor
      super().__init__(frequency_bins, time_bins, **kwargs)

  def call(self, magnitude):
    magnitude = tf.ensure_shape(magnitude, [None, self.time_bins, self.frequency_bins])
    attn_output = self.mha(
        query=magnitude,  # The querys is what you're trying to find.
        key=magnitude,  # The keys what sort of information the dictionary has.
        value=magnitude # The value is that information.
        )
    # Cache the attention scores for plotting later.
    #self.last_attn_scores = attn_scores

    x = self.add([magnitude, attn_output])
    x = self.layernorm(x)
    #x = tf_round(x, 4)
    return x

In [ ]:
sample_lsa = SelfAttention(frequency_bins=sample_frequency_bins[sample_index_for_testing],
                           time_bins=sample_time_bins[sample_index_for_testing],
                           num_heads=sample_num_heads[sample_index_for_testing],
                           dropout=sample_dropout_rate,
                           kernel_regularizer=l2(sample_regularizer_rate),
                           bias_regularizer=l2(sample_regularizer_rate),
                           activity_regularizer=l2(sample_regularizer_rate)
                                    )

In [ ]:
#sample_lsa.build(out_sft_mag.shape)
out_lsa_mag = sample_lsa(out_sft_mag)
print(out_sft_mag.shape)
print(out_lsa_mag.shape)

(3, 3, 33)
(3, 3, 33)


In [ ]:
out_lsa_mag[0,0,:]

<tf.Tensor: shape=(33,), dtype=float32, numpy=
array([ 1.1790699 ,  0.564249  ,  0.46658126,  0.8237049 , -0.55374366,
       -1.7632929 ,  0.24274501,  0.6227821 , -1.6963226 , -0.5257631 ,
       -0.85958475,  0.31143486, -0.03725466, -1.2568933 , -0.5649503 ,
       -0.6795536 ,  1.3415991 , -1.2393972 , -2.1186123 ,  0.5074447 ,
        0.35048607,  0.09532427,  0.32730228, -1.0697291 ,  0.79557425,
        0.735674  ,  0.2530973 , -1.1999329 ,  1.543486  ,  2.2464192 ,
        0.7137263 ,  0.23510481,  0.20922218], dtype=float32)>

## The global cross-attention layer

The similarity (or dot product) between the Query and each Key is computed to determine an attention score. These scores measure how relevant each item (Key) is to the current item (Query).

In [ ]:
class GlobalCrossAttention(BaseAttention):
  def __init__(self, frequency_bins, time_bins, **kwargs):
      # Call the parent class (BaseAttention) constructor
      super().__init__(frequency_bins, time_bins, **kwargs)

  def call(self, magnitude, angle):
    magnitude = tf.ensure_shape(angle, [None, self.time_bins, self.frequency_bins])
    angle = tf.ensure_shape(angle, [None, self.time_bins, self.frequency_bins])
    attn_output, attn_scores = self.mha(
        query=magnitude,  # The querys is what you're trying to find.
        key=angle,  # The keys what sort of information the dictionary has.
        value=angle, # The value is that information.
        return_attention_scores=True
        )

    # Cache the attention scores for plotting later.
    self.last_attn_scores = attn_scores

    x = self.add([magnitude, attn_output])
    x = self.layernorm(x)
    #x = tf_round(x, 4)
    return x

In [ ]:
sample_gca = GlobalCrossAttention(frequency_bins=sample_frequency_bins[sample_index_for_testing],
                                  time_bins=sample_time_bins[sample_index_for_testing],
                                  num_heads=sample_num_heads[sample_index_for_testing],
                                  kernel_regularizer=l2(sample_regularizer_rate),
                                  bias_regularizer=l2(sample_regularizer_rate),
                                  activity_regularizer=l2(sample_regularizer_rate)
                                  )
#sample_gca.build(out_lsa_ang.shape)

In [ ]:
out_gca_mag = sample_gca(magnitude=out_lsa_mag, angle=out_lsa_mag)
print(out_lsa_mag.shape)
print(out_gca_mag.shape)

(3, 3, 33)
(3, 3, 33)


In [ ]:
out_gca_mag[0,0,:]

<tf.Tensor: shape=(33,), dtype=float32, numpy=
array([ 2.204545  , -0.85885155, -0.45152178, -0.37757188, -0.16481535,
       -0.22591953, -0.20017585, -0.59517705,  0.03543307, -1.0487615 ,
       -2.359147  ,  0.3629164 ,  0.7353787 , -0.7167132 ,  0.7138909 ,
        0.2266638 ,  1.2048559 ,  0.6508487 ,  0.23890111, -0.52623606,
        0.68161565, -1.2068477 , -1.0051154 ,  1.1889529 , -1.2217807 ,
       -1.4504949 , -0.977343  ,  0.84239143,  2.0846906 ,  0.27080092,
        0.25128853,  0.38198295,  1.311316  ], dtype=float32)>

## Gated - Feed Forward Network Layer

In [ ]:
class FeedForward(Layer):
  # dff - dence feed forward neurons
  # sft_len - output shape Fourier Transform
  def __init__(self, frequency_bins, time_bins, regularizer_rate, dropout_rate=0.1):
    super().__init__()
    self.seq = Sequential([
        #Dense(tf.get_static_value(tf.cast(tf.math.sqrt(tf.cast(frequency_bins, dtype=tf.bfloat16))*time_bins, dtype=tf.int32)),
        #Dense(tf.get_static_value(tf.cast(frequency_bins*time_bins, dtype=tf.int32)),
        #      kernel_regularizer=l2(regularizer_rate),
        #      bias_regularizer=l2(regularizer_rate),
        #      activity_regularizer=l2(regularizer_rate),
        #      activation='relu', kernel_initializer=initializer_for_relu),
        #Dropout(dropout_rate),

        #Dense(tf.get_static_value(tf.cast(frequency_bins*time_bins, dtype=tf.int32)),
              #kernel_regularizer=l2(regularizer_rate),
              #bias_regularizer=l2(regularizer_rate),
              #activity_regularizer=l2(regularizer_rate),
        #      activation='relu', kernel_initializer=initializer_for_relu),
        #Dropout(dropout_rate),

        Dense(tf.get_static_value(tf.cast(frequency_bins, dtype=tf.int32)),
              #kernel_regularizer=l2(regularizer_rate),
              #bias_regularizer=l2(regularizer_rate),
              #activity_regularizer=l2(regularizer_rate),
              activation='relu', kernel_initializer=initializer_for_relu),
        Dropout(dropout_rate),

        Dense(frequency_bins,
              #kernel_regularizer=l2(regularizer_rate),
              #bias_regularizer=l2(regularizer_rate),
              #activity_regularizer=l2(regularizer_rate)
              activation='sigmoid', kernel_initializer=initializer_for_sigmoid
              ) # NO Activation Function, to predict linear values as given in original paper
    ])
    #self.add = Add()
    self.multiply = Multiply()
    self.layer_norm = LayerNormalization()

  def call(self, x):
    x = self.multiply([x, self.seq(x)])
    x = self.layer_norm(x)
    return x

In [ ]:
sample_ffn = FeedForward(frequency_bins=sample_frequency_bins[sample_index_for_testing],
                         time_bins=sample_time_bins[sample_index_for_testing],
                         regularizer_rate=sample_regularizer_rate)
sample_ffn_out = sample_ffn(out_gca_mag)
print(out_gca_mag.shape)
print(sample_ffn_out.shape)

(3, 3, 33)
(3, 3, 33)


In [ ]:
sample_ffn_out[0,0,:]

<tf.Tensor: shape=(33,), dtype=float32, numpy=
array([ 2.972637  , -0.9075121 , -0.577577  , -0.44822773, -0.18780813,
       -0.21463193, -0.04395513, -0.0950717 ,  0.01280708, -1.4200058 ,
       -2.47751   ,  0.39532605,  0.20715506, -0.93244165,  0.30269724,
        0.12366861,  0.27923468,  0.2338658 ,  0.08032132, -0.29372144,
        0.16993427, -0.56609124, -0.4504433 ,  1.693913  , -0.4870661 ,
       -1.2207144 , -1.0195692 ,  0.8972387 ,  2.2250485 ,  0.10393256,
        0.24971868,  0.47883195,  0.91601634], dtype=float32)>

## Complete Encoder Layer

In [ ]:
class EncoderLayer(Layer):
  def __init__(self,*, frequency_bins, time_bins, num_heads, dropout_rate, regularizer_rate):
    super().__init__()

    self.self_attention = SelfAttention(frequency_bins=frequency_bins,
                                        time_bins=time_bins,
                                        num_heads=num_heads,
                                        dropout=dropout_rate
                                        #kernel_regularizer=l2(regularizer_rate),
                                        #bias_regularizer=l2(regularizer_rate),
                                        #activity_regularizer=l2(regularizer_rate)
                                             )

    #self.ffn = FeedForward(frequency_bins=frequency_bins, time_bins=time_bins, regularizer_rate=regularizer_rate, dropout_rate=dropout_rate)

  def call(self, angle):
    angle = self.self_attention(angle)
    #angle = self.ffn(angle)
    return angle

In [ ]:
sample_encoder_layer = EncoderLayer(frequency_bins=sample_frequency_bins[sample_index_for_testing],
                                    time_bins=sample_time_bins[sample_index_for_testing],
                                    num_heads=sample_num_heads[sample_index_for_testing],
                                    dropout_rate=sample_dropout_rate,
                                    regularizer_rate=sample_regularizer_rate)
#sample_encoder_layer.build(out_sft_ang.shape)

In [ ]:
sample_encoder_output = sample_encoder_layer(out_sft_ang, training=False)
# Print the shape.
print(out_sft_ang.shape)
print(sample_encoder_output.shape)

(3, 3, 33)
(3, 3, 33)


In [ ]:
class Encoder(Layer):
  def __init__(self, *, frequency_bins, time_bins, num_layers, num_heads, dropout_rate, regularizer_rate):
    super().__init__()

    self.num_layers=num_layers

    self.enc_layers = [ EncoderLayer(frequency_bins=frequency_bins,
                                     time_bins=time_bins,
                                     num_heads=num_heads,
                                     dropout_rate=dropout_rate,
                                     regularizer_rate=regularizer_rate
                                     ) for _ in range(num_layers)]

  def call(self, angle):
    for i in range(self.num_layers):
      angle = self.enc_layers[i](angle)

    #self.last_attn_scores = self.enc_layers[-1].last_attn_scores
    return angle

In [ ]:
# Testing the encoder

# Instantiate the encoder.
sample_encoder = Encoder(frequency_bins=sample_frequency_bins[sample_index_for_testing],
                         time_bins=sample_time_bins[sample_index_for_testing],
                         num_layers=sample_num_layers[sample_index_for_testing],
                         num_heads=sample_num_heads[sample_index_for_testing],
                         dropout_rate=sample_dropout_rate,
                         regularizer_rate=sample_regularizer_rate)
#sample_encoder.build(out_sft_ang.shape)

In [ ]:
sample_encoder_output = sample_encoder(out_sft_ang, training=False)
# Print the shape.
print(out_sft_ang.shape)
print(sample_encoder_output.shape)

(3, 3, 33)
(3, 3, 33)


In [ ]:
sample_encoder_output[0,0,:]

<tf.Tensor: shape=(33,), dtype=float32, numpy=
array([-0.7167841 , -0.42196512,  2.286218  , -0.9100698 ,  0.9723666 ,
        1.5574119 , -0.05303797,  2.159482  , -1.1081222 , -0.7488479 ,
       -1.3660975 ,  0.9924534 , -0.5357036 , -1.1316019 , -0.7606728 ,
       -0.08101504, -1.062786  ,  0.5154885 ,  0.3009772 ,  1.0953786 ,
       -0.8357161 ,  1.118093  , -2.008595  ,  1.0043137 ,  0.5482438 ,
       -0.28600633,  0.05738601, -0.36398855, -0.6585642 ,  0.25185263,
        0.5283468 ,  0.27403864, -0.61247647], dtype=float32)>

## Complete Decoder Layer

Each DecoderLayer containing a CausalSelfAttention, a CrossAttention, and a FeedForward layer:

In [ ]:
class DecoderLayer(Layer):
  def __init__(self, *, frequency_bins, time_bins, num_heads, dropout_rate, regularizer_rate):
    super().__init__()

    self.local_self_att = SelfAttention(
        frequency_bins=frequency_bins,
        time_bins=time_bins,
        num_heads=num_heads,
        dropout=dropout_rate
        #kernel_regularizer=l2(regularizer_rate),
        #bias_regularizer=l2(regularizer_rate),
        #activity_regularizer=l2(regularizer_rate)
    )

    self.global_cross_att = GlobalCrossAttention(
        frequency_bins=frequency_bins,
        time_bins=time_bins,
        num_heads=num_heads,
        dropout=dropout_rate
        #kernel_regularizer=l2(regularizer_rate),
        #bias_regularizer=l2(regularizer_rate),
        #activity_regularizer=l2(regularizer_rate)
    )

    self.ffn = FeedForward(frequency_bins=frequency_bins,
                           time_bins=time_bins,
                           regularizer_rate=regularizer_rate,
                           dropout_rate=dropout_rate)

  def call(self, magnitude, angle):
    self_att_out = self.local_self_att(magnitude)
    cross_att_out = self.global_cross_att(magnitude=magnitude, angle=self_att_out)

    # Cache the last attention scores for plotting later
    #self.last_attn_scores = self.global_cross_att.last_attn_scores

    fnn_out = self.ffn(cross_att_out)

    return fnn_out

In [ ]:
# Testing decoder layer
sample_decoder_layer = DecoderLayer(frequency_bins=sample_frequency_bins[sample_index_for_testing],
                                    time_bins=sample_time_bins[sample_index_for_testing],
                                    num_heads=sample_num_heads[sample_index_for_testing],
                                    dropout_rate=sample_dropout_rate,
                                    regularizer_rate=sample_regularizer_rate)
#sample_decoder_layer.build(out_sft_mag.shape)
sample_decoder_layer_output = sample_decoder_layer(magnitude=out_sft_mag, angle=sample_encoder_output)

In [ ]:
print(out_sft_mag.shape)
print(sample_decoder_layer_output.shape)

(3, 3, 33)
(3, 3, 33)


In [ ]:
sample_decoder_layer_output[0,0,:]

<tf.Tensor: shape=(33,), dtype=float32, numpy=
array([-0.49825454,  0.47121042,  0.8353162 , -0.02557037,  0.90962774,
        1.4106195 , -0.17256825, -1.0789157 , -0.6469418 , -1.5278851 ,
        0.76014644,  0.27503917,  1.582253  , -1.2358878 ,  2.4050727 ,
       -1.0069855 ,  0.59519464,  1.4345629 , -0.9452836 , -1.1054679 ,
        0.4670615 ,  0.45530385,  0.44157588, -0.46929547, -0.5257332 ,
       -0.48534143, -1.4979775 ,  0.4452047 ,  0.7480116 , -0.41173637,
       -1.4862555 ,  0.98090476, -1.0970045 ], dtype=float32)>

In [ ]:
class Decoder(Layer):
  def __init__(self, *, frequency_bins, time_bins, num_heads, num_layers, regularizer_rate, dropout_rate=0.1):
    super().__init__()

    self.num_layers=num_layers

    self.decoder_layer = [ DecoderLayer(frequency_bins=frequency_bins,
                                        time_bins=time_bins,
                                        num_heads=num_heads,
                                        dropout_rate=dropout_rate,
                                        regularizer_rate=regularizer_rate
                                        ) for _ in range(num_layers)]

  def call(self, magnitude, angle):
    for i in range(self.num_layers):
      magnitude  = self.decoder_layer[i](magnitude, angle)

    #self.last_attn_scores = self.decoder_layer[-1].last_attn_scores

    return magnitude

In [ ]:
# Test the decoder

# Instantiate the decoder.
sample_decoder = Decoder(frequency_bins=sample_frequency_bins[sample_index_for_testing],
                         time_bins=sample_time_bins[sample_index_for_testing],
                         num_layers=sample_num_layers[sample_index_for_testing],
                         num_heads=sample_num_heads[sample_index_for_testing],
                         dropout_rate=sample_dropout_rate,
                         regularizer_rate=sample_regularizer_rate)
#sample_decoder.build(out_sft_mag.shape)
output = sample_decoder(magnitude=out_sft_mag, angle=sample_encoder_output)

In [ ]:
output[0,0,:]

<tf.Tensor: shape=(33,), dtype=float32, numpy=
array([ 1.0508031 ,  0.37066197,  0.3840768 ,  0.11385274,  0.6825486 ,
        1.4450414 , -0.86724895,  0.17507938,  0.15849046, -0.2240977 ,
       -0.37651077, -0.42264518,  0.16557714,  0.22568738, -0.35435817,
       -2.1344504 , -0.16760898, -0.13901545,  0.60649496,  2.204105  ,
       -0.26274082, -0.2162578 , -2.895268  , -0.51888984, -0.22346027,
        0.2813723 ,  2.0997322 , -1.0494118 , -0.27285787, -0.79873556,
       -0.5448674 ,  1.5289699 , -0.02406857], dtype=float32)>

In [ ]:
print(out_sft_mag.shape)
print(sample_encoder_output.shape)
print(output.shape)

(3, 3, 33)
(3, 3, 33)
(3, 3, 33)


## Transformer

In [ ]:
class TransformerLayer(Layer):
  def __init__(self, *, signal_length, frame_length, frame_step,
               max_signal_length, min_signal_length,
               #frequency_bins, time_bins,
               num_heads, num_layers, regularizer_rate, dropout_rate=0.1):
    super().__init__()

    self.time_bins = tf.cast(((signal_length - frame_length)/frame_step)+1 , dtype=tf.int32)

    self.frequency_bins = tf.cast(tf.math.floor((frame_length/2) +1), dtype=tf.int32)

    self.filter_size = tf.cast(tf.math.floor(max_signal_length/2), dtype=tf.int32)

    self.sft_layer = FourierTransform(signal_length=signal_length,
                                      frame_length=frame_length,
                                      frame_step=frame_step)

    self.encoder = Encoder(frequency_bins=tf.get_static_value(self.frequency_bins),
                           time_bins=tf.get_static_value(self.time_bins),
                           num_layers=num_layers,
                           num_heads=num_heads,
                           dropout_rate=dropout_rate,
                           regularizer_rate=regularizer_rate)

    self.decoder = Decoder(frequency_bins=tf.get_static_value(self.frequency_bins),
                           time_bins=tf.get_static_value(self.time_bins),
                           num_layers=num_layers,
                           num_heads=num_heads,
                           dropout_rate=dropout_rate,
                           regularizer_rate=regularizer_rate)

    self.flatten_layer = Flatten()

    self.dense_layer = Dense(min_signal_length,
                             #tf.get_static_value(self.filter_size),
                              activation='relu', kernel_initializer=initializer_for_relu
                              #kernel_regularizer=l2(regularizer_rate),
                              #bias_regularizer=l2(regularizer_rate),
                              #activity_regularizer=l2(regularizer_rate)
                             )

    #self.conv_layer = Conv1D(filters=self.filter_size,
    #                         kernel_size=[tf.get_static_value(self.time_bins)],
    #                         strides=[tf.get_static_value(self.time_bins)],
    #                         #activation='relu', kernel_initializer=initializer_for_relu,
    #                         input_shape = (tf.get_static_value(self.time_bins),tf.get_static_value(self.frequency_bins)))

  def call(self, inputs):
    # To use a Keras model with `.fit` you must pass all your inputs in the
    # first argument.
    magnitude_x, angle_x = self.sft_layer(inputs)

    enc_out = self.encoder(angle_x)  # (batch_size, time_bins, frequency_bins)

    dec_out = self.decoder(magnitude_x, enc_out)  # (batch_size, time_bins, frequency_bins)

    # Reduce the dimentionality.
    #dec_out = self.conv_layer(dec_out)

    # Final linear layer output.
    dec_out = self.flatten_layer(dec_out) # (batch_size, time_bins * frequency_bins)

    # Reduce the dimentionality.
    dec_out = self.dense_layer(dec_out)
    # Return the output and the attention weights.
    return dec_out


In [ ]:
sample_transformer_layer = TransformerLayer(signal_length=sample_signal_length[sample_index_for_testing],
                                            max_signal_length=sample_max_signal_length,
                                            min_signal_length=sample_min_signal_length,
                                            frame_length=sample_frame_length[sample_index_for_testing],
                                            frame_step=sample_frame_step[sample_index_for_testing],
                                            num_layers=sample_num_layers[sample_index_for_testing],
                                            num_heads=sample_num_heads[sample_index_for_testing],
                                            dropout_rate=sample_dropout_rate,
                                            regularizer_rate=sample_regularizer_rate)

sample_transformer_layer_output = sample_transformer_layer(X_train_f[:3])

In [ ]:
sample_transformer_layer_output.shape

TensorShape([3, 16])

## Inception Attention

* tf.ensure_shape Updates the shape of a tensor and checks at runtime that the shape holds.
* When executed, this operation asserts that the input tensor x's shape is compatible with the shape argument.

In [ ]:
class InceptionAttention(Layer):
  def __init__(self,*, frequency_bins, time_bins, num_heads, dropout_rate, regularizer_rate):
    super().__init__()

    #self.frequency_bins_val = (tf.reduce_sum(frequency_bins))

    self.self_attention = SelfAttention(frequency_bins=frequency_bins,
                                        time_bins=time_bins,
                                        num_heads=num_heads,
                                        dropout=dropout_rate,
                                        kernel_regularizer=l2(regularizer_rate),
                                        bias_regularizer=l2(regularizer_rate),
                                        activity_regularizer=l2(regularizer_rate)
                                        )

    #self.ffn = FeedForward(frequency_bins=frequency_bins,
    #                       time_bins=time_bins,
    #                       regularizer_rate=regularizer_rate,
    #                       dropout_rate=dropout_rate)

    #self.dense_layer = Dense(filter_size,
    #                         activation='relu', kernel_initializer=initializer_for_relu
    #                         #kernel_regularizer=l2(regularizer_rate),
    #                         #bias_regularizer=l2(regularizer_rate),
    #                         #activity_regularizer=l2(regularizer_rate)
    #                         )

  def call(self, magnitude):
    magnitude = self.self_attention(magnitude)
    #magnitude = self.ffn(magnitude)
    #magnitude = self.dense_layer(magnitude)
    return magnitude

## Pre-Final FFN Layer

In [ ]:
class PreFinalFeedForward(Layer):
  # dff - dence feed forward neurons
  # sft_len - output shape Fourier Transform
  def __init__(self, min_signal_length, dropout_rate=0.1):
    super().__init__()
    self.seq = Sequential([
        Dense(1),
        Dense(min_signal_length), # NO Activation Function, to predict linear values as given in original paper
        Dropout(dropout_rate)
    ])

  def call(self, x):
    x = self.seq(x)
    return x

In [ ]:
sample_pre_ffn = PreFinalFeedForward(min_signal_length=sample_min_signal_length)
sample_pre_ffn_out = sample_pre_ffn(out_gca_mag)
print(out_gca_mag.shape)
print(sample_pre_ffn_out.shape)

(3, 3, 33)
(3, 3, 16)


In [ ]:
sample_pre_ffn_out[0,0,:]

<tf.Tensor: shape=(16,), dtype=float32, numpy=
array([-0.2151384 , -0.25789186, -0.06548694, -0.31111208,  0.09293319,
       -0.2572269 , -0.10579755, -0.07363471,  0.23473677,  0.3647576 ,
        0.18603444,  0.19519892, -0.01273074,  0.29509613,  0.08502354,
       -0.33132443], dtype=float32)>

## Inception

In [ ]:
class Inception(Model):
  def __init__(self, *, signal_length, frame_length, frame_step,
               max_signal_length, min_signal_length,
              #frequency_bins, time_bins,
              num_heads, num_layers, regularizer_rate, dropout_rate=0.1):
    super().__init__()
    self.iteration_len = tf.cast(tf.get_static_value(tf.size(frame_length)), dtype=tf.int32)

    self.signal_length = signal_length
    self.frequency_bins = tf.cast(max_signal_length*self.iteration_len/2, dtype=tf.int32)

    self.transformer = [ TransformerLayer(signal_length=signal_length[i], frame_length=frame_length[i], frame_step=frame_step[i],
                                          max_signal_length=max_signal_length, min_signal_length=min_signal_length,
                                          num_heads=num_heads[i], num_layers=num_layers[i], regularizer_rate=regularizer_rate,
                                          dropout_rate=dropout_rate
                                          ) for i in range(self.iteration_len)]

    self.ffn = FeedForward(frequency_bins=1, #
                           time_bins=1,
                           regularizer_rate=regularizer_rate,
                           dropout_rate=dropout_rate)

    self.dropout_layer = Dropout(dropout_rate)

    self.ffn_2 = PreFinalFeedForward(min_signal_length=min_signal_length, dropout_rate=dropout_rate)

    self.inception_att_layer = InceptionAttention(frequency_bins=min_signal_length, #tf.get_static_value(self.frequency_bins),
                                                  time_bins=1,
                                                  num_heads=1,
                                                  dropout_rate=dropout_rate,
                                                  regularizer_rate=regularizer_rate
                                                  )

    self.flat_layer = Flatten()

    self.final_layer = Dense(1) # STUPID !!!!!! - using activation='relu' will limit the output between 0 and infinity, it won't give -ve outputs

  def call(self, inputs):
  # To use a Keras model with `.fit` you must pass all your inputs in the first argument.
    concat_out = inputs
    for i in range(tf.get_static_value(self.iteration_len)):
      sig_len = tf.get_static_value(self.signal_length[i])
      inputs = inputs[:,-sig_len:]
      transformer_out = self.transformer[i](inputs)
      if i == 0:
        concat_out = transformer_out
      else:
        concat_out = tf.concat([concat_out, transformer_out], 1)

    # Gated FFN
    concat_out = self.ffn(concat_out) # (batch_size, min_signal_length * iteration_len)

    concat_out = self.dropout_layer(concat_out)

    # Reduce the input shape
    concat_out = self.ffn_2(concat_out) # (batch_size, min_signal_length)

    # Convert 2D to 3D dataset
    concat_out = tf.expand_dims(concat_out, 1) # (batch_size, 1, min_signal_length)

    concat_out = self.inception_att_layer(concat_out) # (batch_size, 1, min_signal_length)

    # Convert 3D to 2D dataset
    concat_out = self.flat_layer(concat_out)  # (batch_size, min_signal_length)

    logits = self.final_layer(concat_out)  # (batch_size, target_len)

    try:
      # Drop the keras mask, so it doesn't scale the losses/metrics.
      del logits._keras_mask
    except AttributeError:
      pass

    # Return the final output and the attention weights.
    return logits

In [ ]:

sample_transformer = Inception(signal_length=sample_signal_length,
                               frame_length=sample_frame_length,
                               frame_step=sample_frame_step,
                               max_signal_length=sample_max_signal_length,
                               min_signal_length=sample_min_signal_length,
                               num_layers=sample_num_layers,
                               num_heads=sample_num_heads,
                               dropout_rate=sample_dropout_rate,
                               regularizer_rate=sample_regularizer_rate)

sample_transformer_output = sample_transformer(X_train_f[:3])
sample_transformer_output


<tf.Tensor: shape=(3, 1), dtype=float32, numpy=
array([[-0.68390733],
       [-0.6845496 ],
       [-0.6733744 ]], dtype=float32)>

# Model 1

## Model building

In [ ]:
#Define a model with Transformer layer
tf.keras.backend.clear_session()
#model = tf.keras.Sequential()

In [ ]:
# Hyperparameters for Fourier Transform
signal_length = [64, 64, 64, 64, 64, 64, 32] # Length of the input time series
frame_length = [32, 32, 16, 16, 16, 8, 16] # Hop size for Fourier Transform
frame_step = [16, 8, 8, 4, 2, 4, 2] # Hop size for Fourier Transform
inception_lyr_count = len(frame_length)
max_signal_length = max(signal_length)
min_signal_length = min(signal_length)

# Hyperparameters for Attention Layer and DNN Layer
num_layers = [1, 1, 1, 1, 1, 1, 1] # number of TransformerEncoderLayer layers (Original paper = 6)
num_heads = [1, 1, 1, 1, 1, 1, 1] # number of self-attention heads in the MultiheadAttention layer (Original paper = 8)
dropout_rate = 0.3 # Dropout rate
regularizer_rate = 0.0001  # Use to regularizer the weights in attention model

In [ ]:
# The STFT output shape can be defined as:
# Output Shape = (batch_size, 𝐹, 𝑇)
time_bins = list()
frequency_bins = list()
for i in np.arange(inception_lyr_count):
  print(i)
  # Number of Time Frames (T)
  time_bins.append(int(((signal_length[i] - frame_length[i])/frame_step[i] )+1 ))
  print('Time Bins =',time_bins[i])

  # Number of Frequency Bins (F)
  frequency_bins.append(int(tf.math.floor((frame_length[i]/2) +1)))
  print('Frequency Bins =',frequency_bins[i])

  print('output_shape=[batch_size, time_bins, frequency_bins]')
  print('Output shape = (batch_size,',time_bins[i],',',frequency_bins[i],')\n')

0
Time Bins = 3
Frequency Bins = 17
output_shape=[batch_size, time_bins, frequency_bins]
Output shape = (batch_size, 3 , 17 )

1
Time Bins = 5
Frequency Bins = 17
output_shape=[batch_size, time_bins, frequency_bins]
Output shape = (batch_size, 5 , 17 )

2
Time Bins = 7
Frequency Bins = 9
output_shape=[batch_size, time_bins, frequency_bins]
Output shape = (batch_size, 7 , 9 )

3
Time Bins = 13
Frequency Bins = 9
output_shape=[batch_size, time_bins, frequency_bins]
Output shape = (batch_size, 13 , 9 )

4
Time Bins = 25
Frequency Bins = 9
output_shape=[batch_size, time_bins, frequency_bins]
Output shape = (batch_size, 25 , 9 )

5
Time Bins = 15
Frequency Bins = 5
output_shape=[batch_size, time_bins, frequency_bins]
Output shape = (batch_size, 15 , 5 )

6
Time Bins = 9
Frequency Bins = 9
output_shape=[batch_size, time_bins, frequency_bins]
Output shape = (batch_size, 9 , 9 )



In [ ]:
transformer = Inception(signal_length=signal_length,
                        frame_length=frame_length,
                        frame_step=frame_step,
                        max_signal_length=max_signal_length,
                        min_signal_length=min_signal_length,
                        num_layers=num_layers,
                        num_heads=num_heads,
                        dropout_rate=dropout_rate,
                        regularizer_rate=regularizer_rate)

In [ ]:
#transformer.build(X_train_f.shape)

In [ ]:
transformer_output = transformer(X_train_f[:3])

print(X_train_f[:3].shape)
print(transformer_output.shape)

(3, 128)
(3, 1)


In [ ]:
transformer.summary()

Model: "inception"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 transformer_layer (Transfo  multiple                  3037      
 rmerLayer)                                                      
                                                                 
 transformer_layer_1 (Trans  multiple                  4159      
 formerLayer)                                                    
                                                                 
 transformer_layer_2 (Trans  multiple                  2553      
 formerLayer)                                                    
                                                                 
 transformer_layer_3 (Trans  multiple                  4335      
 formerLayer)                                                    
                                                                 
 transformer_layer_4 (Trans  multiple                  78

## Training

In [ ]:
# Define the learning rate variable
optimizer = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.98, epsilon=1e-8, clipnorm=1.0)

In [ ]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, min_lr=0.00001, verbose=2)

In [ ]:
transformer.compile(
    loss='mean_squared_error',#'mean_absolute_error',
    optimizer= optimizer,
    metrics=['R2Score'])# 'mean_squared_error'

In [ ]:
path_model = '/content/drive/MyDrive/MyColabProject/Models'
checkpoint_filepath = (path_model+'/Idea2_1SF_nsepy_Model_inp128_out1_fourierTransform_withInception.weights.h5')
print(checkpoint_filepath)

/content/drive/MyDrive/MyColabProject/Models/Idea2_1SF_nsepy_Model_inp128_out1_fourierTransform_withInception.weights.h5


In [ ]:
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True)

early_stopping = EarlyStopping(monitor="val_loss",
                               patience=15,
                               mode="min",
                               restore_best_weights=True)

In [ ]:
history_1 = transformer.fit(X_train_f,y_train,
                            epochs=25, batch_size=512,
                            shuffle = True,
                            validation_data=[X_val_f,y_val],
                            callbacks=[model_checkpoint_callback, early_stopping, reduce_lr])

Epoch 1/25


219/219 [==============================] - 23s 43ms/step - loss: 79.4247 - r2_score: -0.0036 - val_loss: 79.3331 - val_r2_score: -2.0957e-04 - lr: 0.0010
Epoch 2/25
219/219 [==============================] - 7s 32ms/step - loss: 79.2226 - r2_score: -0.0011 - val_loss: 79.3749 - val_r2_score: -7.5376e-04 - lr: 0.0010
Epoch 3/25
219/219 [==============================] - 8s 35ms/step - loss: 79.2024 - r2_score: -0.0010 - val_loss: 79.3225 - val_r2_score: -1.2112e-04 - lr: 0.0010
Epoch 4/25
219/219 [==============================] - 7s 32ms/step - loss: 79.1496 - r2_score: -4.0483e-04 - val_loss: 79.3388 - val_r2_score: -3.2413e-04 - lr: 0.0010
Epoch 5/25
219/219 [==============================] - 7s 32ms/step - loss: 79.1776 - r2_score: -6.8498e-04 - val_loss: 79.3286 - val_r2_score: -1.9562e-04 - lr: 0.0010
Epoch 6/25
219/219 [==============================] - 8s 35ms/step - loss: 79.1597 - r2_score: -5.2059e-04 - val_loss: 79.3132 - val_r2_score: -5.2452e-06 - lr: 0.0010
Epoch 7/25
219

In [ ]:
history_2 = transformer.fit(X_train_f,y_train,
                            initial_epoch=25, epochs=200,
                            batch_size=512, shuffle = True,
                            validation_data=[X_val_f,y_val],
                            callbacks=[model_checkpoint_callback, early_stopping, reduce_lr])

Epoch 26/200
219/219 [==============================] - 7s 32ms/step - loss: 78.5307 - r2_score: 0.0073 - val_loss: 79.0168 - val_r2_score: 0.0037 - lr: 0.0010
Epoch 27/200
219/219 [==============================] - 7s 34ms/step - loss: 78.3635 - r2_score: 0.0095 - val_loss: 78.7890 - val_r2_score: 0.0066 - lr: 0.0010
Epoch 28/200
219/219 [==============================] - 7s 34ms/step - loss: 78.2311 - r2_score: 0.0111 - val_loss: 78.6228 - val_r2_score: 0.0087 - lr: 0.0010
Epoch 29/200
219/219 [==============================] - 7s 34ms/step - loss: 78.0849 - r2_score: 0.0130 - val_loss: 78.4752 - val_r2_score: 0.0105 - lr: 0.0010
Epoch 30/200
219/219 [==============================] - 7s 34ms/step - loss: 77.8752 - r2_score: 0.0156 - val_loss: 78.3212 - val_r2_score: 0.0125 - lr: 0.0010
Epoch 31/200
219/219 [==============================] - 7s 33ms/step - loss: 77.6676 - r2_score: 0.0183 - val_loss: 78.1336 - val_r2_score: 0.0148 - lr: 0.0010
Epoch 32/200
219/219 [==================

In [ ]:
history_3 = transformer.fit(X_train_f,y_train,
                            initial_epoch=200, epochs=500,
                            batch_size=512, shuffle = True,
                            validation_data=[X_val_f,y_val],
                            callbacks=[model_checkpoint_callback, early_stopping, reduce_lr])

Epoch 201/500
219/219 [==============================] - 7s 32ms/step - loss: 46.2955 - r2_score: 0.4150 - val_loss: 56.1370 - val_r2_score: 0.2923 - lr: 0.0010
Epoch 202/500
219/219 [==============================] - 8s 35ms/step - loss: 46.0608 - r2_score: 0.4179 - val_loss: 55.6851 - val_r2_score: 0.2980 - lr: 0.0010
Epoch 203/500
219/219 [==============================] - 7s 32ms/step - loss: 46.0415 - r2_score: 0.4182 - val_loss: 56.0972 - val_r2_score: 0.2928 - lr: 0.0010
Epoch 204/500
219/219 [==============================] - 7s 32ms/step - loss: 46.2395 - r2_score: 0.4157 - val_loss: 55.7872 - val_r2_score: 0.2967 - lr: 0.0010
Epoch 205/500
219/219 [==============================] - 7s 32ms/step - loss: 46.2046 - r2_score: 0.4161 - val_loss: 55.8338 - val_r2_score: 0.2961 - lr: 0.0010
Epoch 206/500
219/219 [==============================] - 7s 32ms/step - loss: 46.2827 - r2_score: 0.4151 - val_loss: 55.7313 - val_r2_score: 0.2974 - lr: 0.0010
Epoch 207/500
219/219 [===========

KeyboardInterrupt: 

In [ ]:
# Train Loss
train_loss = list(history_1.history['loss'])
train_loss.extend(history_2.history['loss'])
#train_loss.extend(history_3.history['loss'])
#train_loss.extend(history_4.history['loss'])

# Validation Loss
val_loss = list(history_1.history['val_loss'])
val_loss.extend(history_2.history['val_loss'])
#val_loss.extend(history_3.history['val_loss'])
#val_loss.extend(history_4.history['val_loss'])

# plot history
plt.plot(np.array(train_loss[5:]), label='train loss')
plt.plot(np.array(val_loss[5:]), label='validation loss')
plt.xlabel("Number of Epochs ")
plt.ylabel("Loss")
plt.grid()
plt.legend()
plt.show()

## Prediction - from Transformer

In [ ]:
predictions = transformer((X_test_f), training=False)

In [ ]:
predictions.shape

TensorShape([16565, 1])

In [ ]:
predictions[0]

<tf.Tensor: shape=(1,), dtype=float32, numpy=array([-1.3515915], dtype=float32)>

In [ ]:
i = 12
print('Predictions = \n',predictions[i])
print('Actuals = \n',y_test[i])

Predictions = 
 tf.Tensor([0.4651361], shape=(1,), dtype=float32)
Actuals = 
 -2.5


### Validating the results

In [ ]:
y_pred = predictions.numpy().reshape(-1)
y_test = y_test.reshape(-1)
print(y_pred.shape)
print(y_test.shape)

(16565,)
(16565,)


In [ ]:
score_mae = mean_absolute_error(y_test, y_pred)
print("The Mean Absolute Error of our Model is {}".format(round(score_mae, 2)))
score_rmse = mean_squared_error(y_test, y_pred)
print("The Root Mean Squared Error of our Model is {}".format(round(score_rmse, 2)))
score_r2 = r2_score(y_test, y_pred)
print("The accuracy of our model is {}%".format(round(score_r2, 2) *100))

The Mean Absolute Error of our Model is 4.93
The Root Mean Squared Error of our Model is 51.13
The accuracy of our model is 35.0%


In [ ]:
fig = go.Figure(data=[go.Ohlc(x=np.arange(0,10000,1),
                    open=y_pred,
                    high=y_pred,
                    low=y_test,
                    close=y_test
                              )])
fig.show()

In [ ]:
'''
fig = go.Figure(data=[go.Ohlc(x=np.arange(0,1000,1),
                    open=y_pred,
                    high=y_pred,
                    low=y_test,
                    close=y_test
                              )])
fig.show()
'''

'\nfig = go.Figure(data=[go.Ohlc(x=np.arange(0,1000,1),\n                    open=y_pred,\n                    high=y_pred,\n                    low=y_test,\n                    close=y_test\n                              )])\nfig.show()\n'

 ## Prediction - from Saved model

In [ ]:
print(checkpoint_filepath)

/content/drive/MyDrive/MyColabProject/Models/Idea3_1SF_nsepy_Model_inp128_out1_fourierTransform_withInception.weights.h5


In [ ]:
transformer.load_weights(checkpoint_filepath)

### Prediction from Pre-trained loaded transformer

In [ ]:
predictions_reconstructed_model = transformer((X_test_f[:609]), training=False)
y_test_redused = y_test[:609]

In [ ]:
i = 1
print('Predictions = \n',predictions_reconstructed_model[i])
print('Actuals = \n',y_test_redused[i])

Predictions = 
 tf.Tensor([33.843525], shape=(1,), dtype=float32)
Actuals = 
 33.850000000000136


### Compare saved model with Actuals

In [ ]:
y_reconstructed_model = predictions_reconstructed_model.numpy().reshape(-1)
y_test_redused = y_test_redused.reshape(-1)
print(y_reconstructed_model.shape)
print(y_test_redused.shape)

(609,)
(609,)


In [ ]:
score_mae = mean_absolute_error(y_test_redused, y_reconstructed_model)
print("The Mean Absolute Error of our Model is {}".format(round(score_mae, 2)))
score_rmse = mean_squared_error(y_test_redused, y_reconstructed_model)
print("The Root Mean Squared Error of our Model is {}".format(round(score_rmse, 2)))
score_r2 = r2_score(y_test_redused, y_reconstructed_model)
print("The accuracy of our model is {}%".format(round(score_r2, 2) *100))

The Mean Absolute Error of our Model is 1.74
The Root Mean Squared Error of our Model is 24.79
The accuracy of our model is 67.0%


In [ ]:
fig = go.Figure(data=[go.Ohlc(x=np.arange(0,10000,1),
                    open=y_reconstructed_model,
                    high=y_reconstructed_model,
                    low=y_test_redused,
                    close=y_test_redused
                              )])
fig.show()

### Exporting the outputs to csv

In [ ]:
data_df = pd.read_csv(path+'/OSF_results.csv')
data_df.columns

Index(['Actuals', 'OSF_Model_1', 'OSF_2'], dtype='object')

In [ ]:
# Uncomment only if you wish to compare the data in excel sheet
data_df = pd.read_csv(path+'/OSF_results.csv')
#data_df.drop(columns=['Unnamed: 0'], axis=1, inplace=True)

# Using DataFrame.insert() to add a column
#data_df.insert(2, "OSF_2", y_reconstructed_model, True)
data_df.insert(2, "OSF_2_better", y_pred, True)

data_df

Actuals  OSF_Model_1  OSF_2_better  OSF_2
0         542          542           542    542
1         509          509           509    509
2         722          562           586    588
3         517          517           517    517
4         525          525           525    525
...       ...          ...           ...    ...
2518      520          520           520    520
2519      500          500           500    500
2520      542          542           542    542
2521      523          523           523    523
2522      514          514           514    514

[2523 rows x 4 columns]

In [ ]:
data_df.to_csv(path+'/OSF_results.csv',index=False)

# Failed Models

# Success Models

In [ ]:
# 1st Best model
# Input formation
'''
data_q1 = data_df.iloc[:1200,:129].copy()
data_q2 = data_df.iloc[:1200,128:257].copy()
data_q3 = data_df.iloc[:1200,256:385].copy()
data_q4 = data_df.iloc[:1200,384:].copy()
data_df_merged = pd.DataFrame()
data_df_merged = pd.concat([pd.DataFrame(data_q1.values),pd.DataFrame(data_q2.values),pd.DataFrame(data_q3.values),pd.DataFrame(data_q4.values)],
                           ignore_index=True)
'''
# Inputs
'''
X Train shape (3240, 128)
Context Train shape (3240, 1)
Y Train shape (3240, 1)
X Validation shape (1080, 128)
Context Validation shape (1080, 1)
Y Validation shape (1080, 1)
X Test shape (480, 128)
Context Test shape (480, 1)
Y Test shape (480, 1)
'''
# Hyperparameters
'''
# Hyperparameters for Fourier Transform
signal_len = X_train.shape[1] # Length of the input time series
frame_length = 128 # window size for Fourier Transform
frame_step = 64 # Hop size for Fourier Transform
fft_length = frame_length

# Hyperparameters for Attention Layer and DNN Layer
num_layers = 2 # number of TransformerEncoderLayer layers (Original paper = 6)
num_heads = 1 # number of self-attention heads in the MultiheadAttention layer (Original paper = 8)
dropout_rate = 0.2 # Dropout rate

Time Bins = 1
Frequency Bins = 65
output_shape=[batch_size, time_bins, frequency_bins]
Output shape = (batch_size, 1 , 65 )
'''
# Weights
'''
Model: "transformer"
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ fourier_transform (FourierTransform) │ ?                           │             128 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ encoder (Encoder)                    │ ?                           │          69,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ decoder (Decoder)                    │ ?                           │         103,870 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (3, 65)                     │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (3, 1)                      │              66 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘
 Total params: 173,224 (676.66 KB)
 Trainable params: 173,096 (676.16 KB)
 Non-trainable params: 128 (512.00 B)
'''

# Output
'''
Test
The Mean Absolute Error of our Model is 1.47
The Root Mean Squared Error of our Model is 20.26
The accuracy of our model is 32.0%